In [1]:
import numpy as np
import matplotlib as plt
import pandas as pd
%matplotlib inline
from matplotlib import rcParams
rcPramas['figure.figsize'] = 10, 6

In [2]:
dataset = pd.read_csv("3098000032-Mar18-Mar19.csv")

In [6]:

df = pd.DataFrame(dataset, columns=['kWh', 'Time_stamp'])


In [7]:
df

,kWh,Time_stamp
0,10.854,2018-03-01 15:30:00
1,10.750,2018-03-01 14:30:00
2,12.325,2018-03-05 09:00:00
3,12.372,2018-03-05 09:30:00
4,12.415,2018-03-05 10:00:00
...,...,...
17457,6781.400,2019-03-31 22:00:00
17458,6781.920,2019-03-31 22:30:00
17459,6782.520,2019-03-31 23:00:00
17460,6783.120,2019-03-31 23:30:00
